<a href="https://colab.research.google.com/github/3000minsu/2023-LG-Aimers/blob/main/4_1_XGboost_with_prameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1.max_depth

 2.max_leaves

 3.colsample_bytree

 4.subsample(row sample)

 5.n_estimators

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

Mounted at /content/drive


In [ ]:
train = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']
train_y_quality = train_df['Y_Quality']
test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')
train


Done.


,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
RF =  XGBRegressor()
xg_parameters ={'max_depth' : [3,4,5,6, 7, 8] , 'n_estimators': [12,24,32], 'max_leaves':[200,255, 300], 'colsample_bytree': [0.5, 0.75, 1],'sub_sample': [0.5, 0.75, 1], 'random_state':[37]}
depth_parameters ={'max_depth' : [3,4,5,6, 7, 8]}
from sklearn import model_selection
grid_search_xg = model_selection.GridSearchCV ( estimator = RF, param_grid = depth_parameters, scoring = 'accuracy', cv = 10 )
grid_search_xg.fit(train, train_y_quality)
best_xg = grid_search_xg.best_estimator_
best_xg



In [ ]:
RF =  XGBRegressor().fit(train_scaled, train_y_quality)
train_y_quality = train_y_quality.values.reshape(-1,1)
print(train_y_quality.shape)
print(train_y)
RF2 = XGBClassifier().fit(train_y_quality, train_y)
print('Done.')

[15:35:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(598, 1)
0      1
1      2
2      1
3      2
4      1
      ..
593    1
594    0
595    0
596    1
597    1
Name: Y_Class, Length: 598, dtype: int64
Done.


In [ ]:

print(train_y_quality.shape)
print(train_y.shape)


(598, 1)
(598,)


In [ ]:
preds = RF.predict(test_scaled)
preds = preds.reshape(-1,1)
real_preds = RF2.predict(preds)
print('Done.')

Done.


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit['Y_Class'] = real_preds

In [ ]:
submit.to_csv('./baseline_submission.csv', index=False)